### mart 판매 데이터를 기반으로 판매액을 예측하시오.
- 제공된 데이터 목록: mart_train.csv (훈련 데이터), mart_test.csv (평가용 데이터)
- 예측할 컬럼: total (총 판매액)
학습용 데이터(mart_train.csv)를 이용하여 총 판매액을 예측하는 모델을 만든 후, 이를 평가용 데이터(mart_test.csv)에 적용하여 얻은 예측값을 다음과 같은 형식의 CSV 파일로 생성하시오.
- 제출 파일은 다음 한 개의 컬럼을 포함해야 합니다.
- pred: 예측된 총 판매액
- 제출 파일명: 'result.csv'
- 제출한 모델의 성능은 RMSE(Root Mean Square Error) 평가지표에 따라 채점한다.
- 제출 CSV 파일명 및 형태: result.csv

~~~
pred
10000
20000
30000
40000
...
~~~

### 답안 제출 참고
- pd.read_csv('result.csv') 로 제출 코드 확인

# 1. 문제정의
- RMSE
- target: total
- 예측 파일명: result.csv
- 컬럼 1개(pred)

# 2. 라이브러리 및 데이터 불러오기

In [53]:
# 데이터 불러오기
import pandas as pd
train = pd.read_csv("mart_train.csv")
test = pd.read_csv("mart_test.csv")
# print(train.shape, test.shape)
# (700, 10) (300, 9)
# print(train.head()) # total이 타겟
#   branch       city customer_type  gender            product_line      total  \
# 0      A     Yangon        Member  Female       Health and beauty  823457.25   
# 1      C  Naypyitaw        Normal  Female  Electronic accessories  120330.00   
# 2      A     Yangon        Normal    Male      Home and lifestyle  510788.25   
# 3      A     Yangon        Member    Male       Health and beauty  733572.00   
# 4      A     Yangon        Normal    Male       Sports and travel  951567.75   

#   payment_method  rating time_of_day  day_name  
# 0        Ewallet     9.1   afternoon  Saturday  
# 1           Cash     9.6     morning    Friday  
# 2    Credit card     7.4   afternoon    Sunday  
# 3        Ewallet     8.4     evening    Sunday  
# 4        Ewallet     5.3     morning    Friday  
# print(test.head()) # total이 타겟
#   branch       city customer_type  gender         product_line payment_method  \
# 0      C  Naypyitaw        Normal  Female  Fashion accessories        Ewallet   
# 1      B   Mandalay        Normal    Male   Food and beverages    Credit card   
# 2      B   Mandalay        Member  Female  Fashion accessories    Credit card   
# 3      B   Mandalay        Member    Male    Health and beauty           Cash   
# 4      B   Mandalay        Member  Female   Home and lifestyle           Cash   

#    rating time_of_day   day_name  
# 0     9.6   afternoon   Thursday  
# 1     4.3     evening  Wednesday  
# 2     5.0     evening  Wednesday  
# 3     9.2     morning     Sunday  
# 4     6.3   afternoon   Saturday  
# print(train.info())
#  0   branch          700 non-null    object 
#  1   city            700 non-null    object 
#  2   customer_type   700 non-null    object 
#  3   gender          700 non-null    object 
#  4   product_line    700 non-null    object 
#  5   total           700 non-null    float64
#  6   payment_method  700 non-null    object 
#  7   rating          700 non-null    float64
#  8   time_of_day     700 non-null    object 
#  9   day_name        700 non-null    object 
# print(train.describe())
#               total      rating
# count  7.000000e+02  700.000000
# mean   4.850780e+05    7.003429
# std    3.643907e+05    1.713078
# min    1.904175e+04    4.000000
# 25%    2.001195e+05    5.500000
# 50%    3.818745e+05    7.000000
# 75%    7.061276e+05    8.425000
# max    1.563975e+06   10.000000
# print(train.describe(include="O"))
#        branch    city customer_type gender       product_line payment_method  \
# count     700     700           700    700                700            700   
# unique      3       3             2      2                  6              3   
# top         A  Yangon        Normal   Male  Sports and travel           Cash   
# freq      236     236           354    356                127            246   

#        time_of_day  day_name  
# count          700       700  
# unique           3         7  
# top        evening  Saturday  
# freq           309       114  
# print(test.describe(include="O"))
#        branch    city customer_type  gender            product_line  \
# count     300     300           300     300                     300   
# unique      3       3             2       2                       6   
# top         A  Yangon        Member  Female  Electronic accessories   
# freq      104     104           155     157                      58   

#        payment_method time_of_day  day_name  
# count             300         300       300  
# unique              3           3         7  
# top           Ewallet     evening  Saturday  
# freq              110         123        50  

# 타겟 분할
target = train.pop('total')
# print(target.hist())
# print(train.shape, test.shape)

# 범주형 변수 처리
# 라벨:  406134.48615467397 200
# 원핫:  399575.05516381294 기본
n_cols = train.select_dtypes(exclude="O").columns
o_cols = train.select_dtypes(include="O").columns
from sklearn.preprocessing import LabelEncoder, StandardScaler

# for col in o_cols:
#     le = LabelEncoder()
#     train[col] = le.fit_transform(train[col])
#     test[col] = le.transform(test[col])

scaler = StandardScaler()
train[n_cols] = scaler.fit_transform(train[n_cols])
test[n_cols] = scaler.transform(test[n_cols])

train = pd.get_dummies(train)
test = pd.get_dummies(train)

# 검증 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size = 0.15, random_state = 2022)

# 모델 평가
from sklearn.metrics import mean_squared_error
def rmse(y_val, pred):
    return (mean_squared_error(y_val, pred)**0.5)

# 모델 
# rmse:  409171.1771066565
# rmse:  406134.48615467397 200
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=2022)
model.fit(X_tr, y_tr)
pred = model.predict(X_val)
print("rmse: ", rmse(y_val, pred))

rmse:  399575.05516381294


In [54]:
pred = model.predict(test)
submit = pd.DataFrame({
    'pred': pred
})
submit.to_csv('작업형2제출.csv', index=False)

In [55]:
df = pd.read_csv('작업형2제출.csv')
df

,pred
0,677086.5150
1,215073.8100
2,524849.3775
3,593633.0925
4,752883.0750
...,...
695,686839.3875
696,315445.0950
697,383131.9800
698,774397.8900
